In [1]:
import torch
from networks import FullyConnected, SPU, Normalization
import os
import time

DEVICE = 'cpu'
INPUT_SIZE = 28

In [2]:
class Args:
    net: str
    spec: str

    def __init__(self, net: str, spec: str):
        self.net = net
        self.spec = spec

In [22]:
def analyze(net: torch.nn.Module, inputs: torch.FloatTensor, eps: float, true_label: int) -> str:
    """Returns "verified" if the returned label is the same for all points in a L-infinite epsilon-ball around an input point, and "not verified" otherwise.

    Args:
        net (torch.nn.Module): [description]
        inputs (torch.FloatTensor): [description]
        eps (float): [description]
        true_label (int): [description]

    Returns:
        [str]: Returns "verified" or "not verified"
    """
    start_time = time.time()

    # Compute lower and upper bounds for all points
    detached_inputs = inputs.detach()
    lower_orig = torch.min(detached_inputs - eps * 1, torch.tensor(0.0)).to(DEVICE)
    upper_orig = torch.max(detached_inputs + eps * 1, torch.tensor(1.0)).to(DEVICE)

    with torch.no_grad():
        outputs = net(inputs)
        low = net(lower_orig)
        upper = net(upper_orig)

    print(f'Low: {low}')
    print(f'High: {upper}')
    print(f'Outputs: {outputs.shape}')
    verified = sum((low[0][true_label] > upper[0]).int()) == true_label
    
    if verified: 
        return True


    for i, layer in enumerate(net.layers, 1):
        if type(layer) is SPU:
            continue
        
        elif type(layer) is Normalization:
            continue
        
        elif type(layer) is torch.nn.Flatten:
            continue

        elif type(layer) is torch.nn.Linear:
            continue
    
    # ???

    # Compare around 
    end_time = time.time()
    
    return 0

    # return (true_label_lx > labels_ux).all()


In [23]:
args = Args(
    net="net0_fc1",
    spec="../test_cases/net0_fc1/example_img0_0.01800.txt"
)

with open(args.spec, 'r') as f:
    lines = [line[:-1] for line in f.readlines()]
    true_label = int(lines[0])
    pixel_values = [float(line) for line in lines[1:]]
    eps = float(args.spec[:-4].split('/')[-1].split('_')[-1])

if args.net.endswith('fc1'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [50, 10]).to(DEVICE)

elif args.net.endswith('fc2'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc3'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 10]).to(DEVICE)

elif args.net.endswith('fc4'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc5'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [
                            100, 100, 100, 100, 10]).to(DEVICE)
else:
    assert False

net.load_state_dict(torch.load('../mnist_nets/%s.pt' %
                    args.net, map_location=torch.device(DEVICE)))

inputs = torch.FloatTensor(pixel_values).view(
    1, 1, INPUT_SIZE, INPUT_SIZE).to(DEVICE)

outs = net(inputs)
pred_label = outs.max(dim=1)[1].item()
assert pred_label == true_label

print(f'Network: {net}')
print(f'Inputs shape: {inputs.shape}')
print(f'Epsilon: {eps}')
print(f'True label: {true_label}')

print()

if analyze(net, inputs, eps, true_label):
    print('verified')
else:
    print('not verified')


Network: FullyConnected(
  (layers): Sequential(
    (0): Normalization()
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=784, out_features=50, bias=True)
    (3): SPU()
    (4): Linear(in_features=50, out_features=10, bias=True)
  )
)
Inputs shape: torch.Size([1, 1, 28, 28])
Epsilon: 0.018
True label: 1

Low: tensor([[-2.6558, -2.8654,  0.3886,  0.1550, -4.0704,  5.2014,  2.0776, -0.8347,
         -4.7754, -5.9432]])
High: tensor([[ -35.3241, -358.8481, -322.7823,  487.9682,  113.4878, -659.5154,
          619.2648,  352.8987,  673.6779, -608.7426]])
Outputs: torch.Size([1, 10])
not verified
